In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from datetime import datetime
import pytz

from kfp import compiler
from google.cloud import aiplatform

from training_pipeline import pipeline

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# Project variables:
PROJECT_ID=os.getenv('PROJECT_ID')
BUCKET_NAME=os.getenv('BUCKET_NAME')
USER=os.getenv('USER')
LOCATION=os.getenv('LOCATION')
SERVICE_ACCOUNT=os.getenv('SERVICE_ACCOUNT')

In [ ]:
compiler.Compiler().compile(pipeline, package_path='pipeline.yaml')

In [ ]:
# Job parameters:
tz = pytz.timezone("US/Eastern")
EXECUTION_TS = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
DISPLAY_NAME = "pipeline-citibike-{}".format(EXECUTION_TS)
PIPELINE_ROOT = "gs://{}/midterm_project/pipeline_root".format(BUCKET_NAME)

# Pipeline input parameters:
data_gcs_uri = f'gs://{BUCKET_NAME}/midterm_project/data/citi_bike_data_00000.csv' # DON'T FORGET TO CREATE A GCS BUCKET AND UPLOAD THE citi_bike_data_00000.csv FILE
label = "percentage_bikes_available"

# Running:
job = aiplatform.PipelineJob(
        display_name=DISPLAY_NAME,
        template_path='pipeline.yaml',
        pipeline_root=PIPELINE_ROOT,
        enable_caching=True,
        project=PROJECT_ID,
        location=LOCATION,
        labels={"user": USER},
        parameter_values={
            "data_gcs_uri": data_gcs_uri,
            "label": label,
        },
)

job.submit(service_account=SERVICE_ACCOUNT)

Creating PipelineJob
PipelineJob created. Resource name: projects/532579765435/locations/us-central1/pipelineJobs/pipeline-citibike-20231106193221
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/532579765435/locations/us-central1/pipelineJobs/pipeline-citibike-20231106193221')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/pipeline-citibike-20231106193221?project=532579765435


In [ ]:
pipeline_df = aiplatform.get_pipeline_df(pipeline="pipeline-citibike")

In [ ]:
pipeline_df.head(1)

,pipeline_name,run_name,param.input:label,param.input:data_gcs_uri
0,pipeline-citibike,pipeline-citibike-20231106192444,percentage_bikes_available,gs://dz-d-stg-us-ml-zoomcamp/midterm_project/d...
